# 서울특별시 공공자전거 따릉이 대여이력 연도별 전처리

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
%ls data/

bike_rent_station.csv
bike_rent_station_gu.csv
seoul-bike-2016.csv
seoul-bike-2017.csv
seoul-bike-2018.csv
서울_공공자전거_따릉이_이용현황자료_20161026.xlsx
서울특별시 공공자전거 대여이력 정보_2015년.csv
서울특별시 공공자전거 대여이력 정보_2016년_1.csv
서울특별시 공공자전거 대여이력 정보_2016년_2.csv
서울특별시 공공자전거 대여이력 정보_2016년_3.csv
서울특별시 공공자전거 대여이력 정보_2017년_1.csv
서울특별시 공공자전거 대여이력 정보_2017년_2분기_1.csv
서울특별시 공공자전거 대여이력 정보_2017년_2분기_2.csv
서울특별시 공공자전거 대여이력 정보_2017년_3분기_1.csv
서울특별시 공공자전거 대여이력 정보_2017년_3분기_2.csv
서울특별시 공공자전거 대여이력 정보_2017년_4분기_1.csv
서울특별시 공공자전거 대여이력 정보_2017년_4분기_2.csv
서울특별시 공공자전거 대여이력 정보_2018년_1분기1.csv
서울특별시 공공자전거 대여이력 정보_2018년_2분기_1.csv
서울특별시 공공자전거 대여이력 정보_2018년_2분기_2.csv
서울특별시 공공자전거 대여이력 정보_2018년_2분기_3.csv
서울특별시 공공자전거 대여이력 정보_2018년_2분기_4.csv
서울특별시 공공자전거 대여정보_201807_01.xlsx
서울특별시 공공자전거 대여정보_201807_02.xlsx
서울특별시 공공자전거 대여정보_201808.xlsx
서울특별시 공공자전거 대여정보_201809_1.xlsx
서울특별시 공공자전거 대여정보_201809_2.xlsx
서울특별시 공공자전거 대여정보_201810_02.xlsx
서울특별시 공공자전거 대여정보_201811.xlsx


In [3]:
def preprocessing(df):
    # 컬럼에 있는 '를 제거한다.
    df.columns = df.columns.str.strip("'")
    # 불필요하게 '가 들어가 있다. 스트링 타입으로 되어있는 데이터에서 '을 제거하자.
    df = df.apply(lambda x: x.str.strip("'") if x.dtype == np.dtype('object') else x)
    return df

In [4]:
# 컬럼명이 2017년을 기점으로 달라지기 때문에 최근 컬럼에 맞게 컬럼명을 변경해 준다.
# 우선 최근 컬럼명을 가져온다.
df_for_columns = pd.read_csv("data/서울특별시 공공자전거 대여이력 정보_2018년_2분기_4.csv", encoding="cp949")
df_for_columns.columns = df_for_columns.columns.str.strip("'")
cols = df_for_columns.columns
len(cols)

11

In [5]:
# df = pd.read_excel('data/서울특별시 공공자전거 대여정보_201807_01.xlsx', encoding='cp949')
# df.head()

In [6]:
def get_file_list_by_year(year):
    file_list = []
    for root, dirs, files in os.walk("data/"):
        for filename in files:
            if filename.startswith('서울특별시 공공자전거 대여이력 정보_{}년'.format(year)):
                file_list.append(filename)
            elif filename.startswith('서울특별시 공공자전거 대여정보_{}'.format(year)):
                file_list.append(filename)
    return sorted(file_list)

get_file_list_by_year(2018)

['서울특별시 공공자전거 대여이력 정보_2018년_1분기1.csv',
 '서울특별시 공공자전거 대여이력 정보_2018년_2분기_1.csv',
 '서울특별시 공공자전거 대여이력 정보_2018년_2분기_2.csv',
 '서울특별시 공공자전거 대여이력 정보_2018년_2분기_3.csv',
 '서울특별시 공공자전거 대여이력 정보_2018년_2분기_4.csv',
 '서울특별시 공공자전거 대여정보_201807_01.xlsx',
 '서울특별시 공공자전거 대여정보_201807_02.xlsx',
 '서울특별시 공공자전거 대여정보_201808.xlsx',
 '서울특별시 공공자전거 대여정보_201809_1.xlsx',
 '서울특별시 공공자전거 대여정보_201809_2.xlsx',
 '서울특별시 공공자전거 대여정보_201810_02.xlsx',
 '서울특별시 공공자전거 대여정보_201811.xlsx']

In [7]:
def concat_data(year):
    df_lists = []
    # 파일별로 데이터를 가져온다.
    files = get_file_list_by_year(year)
    for filename in sorted(files):
        # 파일을 읽어온다. csv와 xlsx 파일을 구분해 불러온다.
        df_temp = read_file(filename, year)
        # 해당 데이터프레임이 비어있지 않다면 병합하기 위해 리스트에 담는다.
        if not df_temp.empty :
            print(filename, df_temp.shape)
            df_lists.append(df_temp)
    # 리스트가 비어있지 않다면 데이터프레임을 합쳐준다.       
    if df_lists :    
        df = pd.concat(df_lists)
    print(df.shape)
    return df        

In [8]:
def read_file(filename, year):
    folder_path = 'data/'
    if filename.startswith('서울특별시 공공자전거 대여이력 정보_{}년'.format(year)):
        # 서울시 정보소통광장에서 다운로드 받은 데이터의 인코딩이 cp949이기 때문에 인코딩을 지정해 주어야 한다.
        df_temp = pd.read_csv(folder_path+filename, encoding='cp949', low_memory=False)
        # 데이터에 불필요한 따옴표(')가 들어가 있다. 제거해 준다.
        df_temp = preprocessing(df_temp)
        # 2017년 중간에 컬럼명이 변경되었다. 2018년 이전 데이터에 대해서 최근 컬럼으로 컬럼명을 변경해 준다. 
        # 최근 컬럼명 cols는 위 셀에서 구한 것을 사용한다.
        if year < 2018 :
            df_temp.columns = cols
    elif filename.startswith('서울특별시 공공자전거 대여정보_{}'.format(year)):
        df_temp = pd.read_excel(folder_path+filename, encoding='cp949')
        df_temp.columns = cols
    else:
        df_temp = pd.DataFrame([])
    return  df_temp

In [9]:
%time df_2016 = concat_data(2016)
print(df_2016.shape)

서울특별시 공공자전거 대여이력 정보_2016년_1.csv (482990, 11)
서울특별시 공공자전거 대여이력 정보_2016년_2.csv (815301, 11)
서울특별시 공공자전거 대여이력 정보_2016년_3.csv (282189, 11)
(1580480, 11)
CPU times: user 9.39 s, sys: 1.15 s, total: 10.5 s
Wall time: 11 s
(1580480, 11)


In [10]:
%time df_2017 = concat_data(2017)
print(df_2017.shape)

서울특별시 공공자전거 대여이력 정보_2017년_1.csv (354731, 11)
서울특별시 공공자전거 대여이력 정보_2017년_2분기_1.csv (764154, 11)
서울특별시 공공자전거 대여이력 정보_2017년_2분기_2.csv (532503, 11)
서울특별시 공공자전거 대여이력 정보_2017년_3분기_1.csv (918911, 11)
서울특별시 공공자전거 대여이력 정보_2017년_3분기_2.csv (880529, 11)
서울특별시 공공자전거 대여이력 정보_2017년_4분기_1.csv (852503, 11)
서울특별시 공공자전거 대여이력 정보_2017년_4분기_2.csv (672623, 11)
(4975954, 11)
CPU times: user 46.9 s, sys: 6.37 s, total: 53.2 s
Wall time: 55.8 s
(4975954, 11)


In [11]:
%time df_2018 = concat_data(2018)
print(df_2018.shape)

서울특별시 공공자전거 대여이력 정보_2018년_1분기1.csv (784734, 11)
서울특별시 공공자전거 대여이력 정보_2018년_2분기_1.csv (677878, 11)
서울특별시 공공자전거 대여이력 정보_2018년_2분기_2.csv (951446, 11)
서울특별시 공공자전거 대여이력 정보_2018년_2분기_3.csv (566859, 11)
서울특별시 공공자전거 대여이력 정보_2018년_2분기_4.csv (586304, 11)
서울특별시 공공자전거 대여정보_201807_01.xlsx (1048575, 11)
서울특별시 공공자전거 대여정보_201807_02.xlsx (41307, 11)
서울특별시 공공자전거 대여정보_201808.xlsx (1026093, 11)
서울특별시 공공자전거 대여정보_201809_1.xlsx (1048575, 11)
서울특별시 공공자전거 대여정보_201809_2.xlsx (385067, 11)
서울특별시 공공자전거 대여정보_201810_02.xlsx (366465, 11)
서울특별시 공공자전거 대여정보_201811.xlsx (958253, 11)
(8441556, 11)
CPU times: user 11min 2s, sys: 17.2 s, total: 11min 19s
Wall time: 11min 41s
(8441556, 11)


In [12]:
df_2016.head()

,자전거번호,대여일시,대여대여소번호,대여대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M)
0,SPB-01338,2016-01-01 0:12,112,극동방송국 앞,4,2016-01-01 0:38,137,NH농협 신촌지점 앞,5,24,4140
1,SPB-00381,2016-01-01 0:22,113,홍대입구역 2번출구 앞,6,2016-01-01 0:47,137,NH농협 신촌지점 앞,9,24,2850
2,SPB-01069,2016-01-01 0:23,211,여의도역 4번출구 옆,11,2016-01-01 0:30,216,삼부아파트1동 앞,10,7,1020
3,SPB-00559,2016-01-01 0:24,113,홍대입구역 2번출구 앞,5,2016-01-01 0:47,137,NH농협 신촌지점 앞,5,22,2850
4,SPB-00492,2016-01-01 0:35,335,종로3가역 15번출구 앞,2,2016-01-01 0:55,343,예일빌딩(율곡로) 앞,1,19,2190


In [13]:
df_2017.head()

,자전거번호,대여일시,대여대여소번호,대여대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M)
0,SPB-00230,2017-01-01 0:00,419,홈플러스 앞,5,2017-01-01 0:21,914,새절역 2번출구,18,20,3340
1,SPB-02803,2017-01-01 0:01,825,서빙고동 주민센터 앞,2,2017-01-01 0:15,822,이촌1동 마을공원,7,14,3380
2,SPB-03956,2017-01-01 0:05,906,연신내역 5번출구150M 아래,1,2017-01-01 0:16,931,역촌파출소,6,11,1610
3,SPB-02864,2017-01-01 0:05,906,연신내역 5번출구150M 아래,9,2017-01-01 0:29,912,응암오거리,1,24,3620
4,SPB-01388,2017-01-01 0:05,330,청계천 한빛광장,6,2017-01-01 0:23,171,임광빌딩 앞,9,17,2280


In [14]:
df_2018.head()

,자전거번호,대여일시,대여대여소번호,대여대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M)
0,SPB-02041,2018-01-01 00:00:10,227,양평2나들목 보행통로 입구,9,2018-01-01 00:04:03,227,양평2나들목 보행통로 입구,9,3,50
1,SPB-03796,2018-01-01 00:00:42,2219,"고속터미널역 8-1번, 8-2번 출구 사이",15,2018-01-01 00:25:41,3511,응봉역 1번출구,9,24,7670
2,SPB-02402,2018-01-01 00:00:58,567,성수역 2번출구 앞,13,2018-01-01 00:09:33,3506,영동대교 북단,2,8,1390
3,SPB-01564,2018-01-01 00:01:02,568,청계8가사거리 부근,5,2018-01-01 00:10:13,521,왕십리역 11번 출구 앞,9,8,1820
4,SPB-05713,2018-01-01 00:01:03,361,동묘앞역 1번출구 뒤,4,2018-01-01 00:06:18,383,신당역 12번 출구 뒤,8,4,850


In [15]:
years = [2016, 2017, 2018]
for year in years:
    df = eval('df_'+str(year))
    df.to_csv('data/seoul-bike-{}.csv'.format(year), index=False)   
    print(df.shape)

(1580480, 11)
(4975954, 11)
(8441556, 11)


In [16]:
# 파일이 제대로 생성되었는지 확인한다.
%ls data/seoul-bike*

data/seoul-bike-2016.csv  data/seoul-bike-2017.csv  data/seoul-bike-2018.csv


In [17]:
# 생성된 파일 안에 내용이 제대로 들어가 있는지 확인한다.
df_2018 = pd.read_csv('data/seoul-bike-2018.csv', low_memory=False)
print(df_2018.shape)
df_2018.head()

(8441556, 11)


,자전거번호,대여일시,대여대여소번호,대여대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M)
0,SPB-02041,2018-01-01 00:00:10,227,양평2나들목 보행통로 입구,9,2018-01-01 00:04:03,227,양평2나들목 보행통로 입구,9,3,50
1,SPB-03796,2018-01-01 00:00:42,2219,"고속터미널역 8-1번, 8-2번 출구 사이",15,2018-01-01 00:25:41,3511,응봉역 1번출구,9,24,7670
2,SPB-02402,2018-01-01 00:00:58,567,성수역 2번출구 앞,13,2018-01-01 00:09:33,3506,영동대교 북단,2,8,1390
3,SPB-01564,2018-01-01 00:01:02,568,청계8가사거리 부근,5,2018-01-01 00:10:13,521,왕십리역 11번 출구 앞,9,8,1820
4,SPB-05713,2018-01-01 00:01:03,361,동묘앞역 1번출구 뒤,4,2018-01-01 00:06:18,383,신당역 12번 출구 뒤,8,4,850
